# Basketball (or Netball) Cannon Toy Problem

Develop a controller for a basketball cannon.  Given the following inputs
1. distance from the basket
2. height from which the ball leaves the cannon
3. speed the ball will leave the cannon

**What angle should the cannon aim the ball such that it will go through the basket without interacting with the backboard or the rim.**

Note that this scenario in actually more similar to Netball than Basketball, since we are neglecting the backboard.
<img src="https://netballamerica.com/wp-content/uploads/1DX28219.jpg">

In [91]:
# import some basic packages
import itertools
import numpy as np
from typing import Tuple

In [92]:
from bokeh import plotting
from bokeh.palettes import Dark2_8 as palette
colors = itertools.cycle(palette)

In [3]:
plotting.output_notebook()

Loading BokehJS ...

In this problem, we observe that the parameters are such that cannon is always aimed in the vertical plane connecting the cannon and the hoop. We can separate the motion of the ball into the horizontal, $x$, and vertical, $y$, directions.  The position in both directions can be written using the linear equations of motions as follows.

$$x(t) = x_0 + \dot{x}_0 t + \frac{1}{2} \ddot{x}_0 t^2$$

$$y(t) = y_0 + \dot{y}_0 t + \frac{1}{2} \ddot{y}_0 t^2$$

We can set the x and y origin at the point where the ball leaves the cannon, so $(x_0, y_0) = (0, 0)$.  The speed at which the ball leaves the cannon is written in terms of x and y as $(\dot{x}_0, \dot{y}_0) = (v\cos\theta, v\sin\theta)$.  The only acceleration in this situation is gravity so $(\ddot{x}_0, \ddot{y}_0) = (0, g)$.  Based on these values, the equations of motion can be written as

$$x(t) = v\cos(\theta) t$$

$$y(t) = v\sin(\theta) t + \frac{1}{2}g t^2$$


To measure how close to the basket each shot is, we want to look at the time, $t_{2}$, when the ball passes the height of the basket, $h$, for the second time.  This is obtained by letting $y(t)=h$ in the second equation

$$\frac{1}{2} g t^2 + v \sin(\theta) t - h = 0$$

then solving for $t$

$$t = \frac{-v \sin(\theta) \pm \sqrt{v^2 \sin^2(\theta)+2gh}}{g}$$

Note that in order to have a reasonable throw, one that forms an arch passing through the rim height twice, the descriminant must be positive, which means

$$v^2 > \frac{-2 g h}{sin^2(\theta)}$$

Lets begin by defining a cannon class that expects the three inputs, distance, height, and speed.  We also define the `fire` method which takes an angle as input then provides as output the distance and impact angle the ball will make with the horizontal plane of the hoop.  We also define a `show_path` method which plots all `fire` attempts.

Note that we are using a ball width of 24 cm, and a rim that is twice that width.  This means that the ball goes through the rim if the center of the ball is within half the ball width of the center of the rim.

In [124]:
class Cannon():
    def __init__(self, dist: float, height: float, speed: float) -> None: 
        self.ball_width = 0.24  # m
        self.rim = dist + self.ball_width * np.array([-1, 1]) / 2  # m
        self.height = 3.05 - height  # delta height from cannon to basket
        self.speed = speed  # m/s
        self.g = -9.8  # m/s^2
        self.path = []
        self.angle = 0
        
    def fire(self, angle: float):
        self.angle = np.deg2rad(angle)
        cos = np.cos(self.angle)
        sin = np.sin(self.angle)

        self.result = 'terrible shot'
        D = (self.speed * sin) ** 2 + 2 * self.g * self.height
        if D > 0:  # ball passes through correct height twice
            self.t_m = (-self.speed * sin - np.sqrt(D)) / self.g  # negative root is the downward part of the arc
            self.x_shot = self.speed * cos * self.t_m  
            if self.rim[0] < self.x_shot < self.rim[1]:
                self.result = 'success'
                self.delta_x = 0
            else:
                self.result = 'miss'
                self.delta_x = np.min(np.abs(self.x_shot - self.rim))
            t_final = self.t_m * 1.1
        else:
            t_final = 1.5  # s
            
        print(f'outcome: {self.result} ({self.delta_x:0.1f} m away)')

        t = np.linspace(0, t_final, 100)  # 0 to 10 s
        x = self.speed * cos * t
        y = self.speed * sin * t + self.g * t**2 / 2
        self.path.append(np.c_[x, y])

        
    def show_path(self):
        p = plotting.figure(
            width=800, height=400,
            match_aspect=True,
        )
        # plot the hoop
        p.circle(self.rim, self.height, fill_alpha=0, radius=0.02, color='orange')
        p.line(self.rim, self.height, color='orange')

        # plot the attempts
        for i, (path, color) in enumerate(zip(self.path, colors)):
            name = f'attempt {i+1}'
            p.line(path[:, 0], path[:, 1], color=color, 
                   legend_label=name, muted_alpha=0.2)
            p.dash(0, 0, angle=self.angle-np.pi/2, color=color)
        p.legend.location = 'top_right'
        p.legend.click_policy = 'mute'
        p.toolbar.autohide = True
        plotting.show(p)
        

Lets look at a sample cannon, that is 5 meters from the basket, 3 meters below the basket, and fires at 8 meters per second.

In [125]:
c0 = Cannon(5, 3, 8)

c0.fire(50)
c0.fire(70)
c0.fire(65)

c0.show_path()

outcome: miss (1.3 m away)
outcome: miss (0.7 m away)
outcome: success (0.0 m away)


We see that the first shot, at an angle of 50 degrees, was 1.3 meters past the basket.
The second shot, at an angle of 70 degrees, was 0.7 meters short of the basket.
The third shot, at an angle of 65 degrees, went through the basket.

## Learn what angle to fire the cannon

Now that we have defined our system, we want to use this cannon to automatically learn what angle to use to make it in the hoop.